# Diamant NV center 
**Het Zeeman-effect waanemen**
practicum handleiding

Versie: december 2024
Door: Rutger Ockhorst, Lodewijk Koopman en Mark Melotto, aanpassingen Silvester Infante Ferreira

Contact docenten: onz@tudelft.nl (noem Rutger Ockhorst)
Contact leerlingen: scholierenlab@tudelft.nl (noem Lennard Duynkerke)

<img src="./media/nv/QDNL.png" width="200" align="left"/>
<br>

Financiering: Quantum Delta NL / Nationaal Groeifonds

<img src="./media/nv/TU_Delft_Logo.svg" width="150" align="right"/>
<br><br>

# 1. Inleiding
In dit experiment bestuderen we de invloed van een extern magnetisch veld op de spintoestand van elektronen. Concreet meten we de fluorescentie van stikstof-gat defecten in diamant(poeder) en de hoeveelheid fluorescentie verandert als we een magneet in de buurt van het poeder houden.

Deze versie van de proef is vooral een kwalitatieve verkenning. Een uitgebreidere versie is in de maak, waarbij met hoge precisie de waarde van het magnetisch veld bepaald kan worden. De defecten in het diamantpoeder zijn een voorbeeld van een quantumsensor. Er is een NLT-module voor havo in de maak, werktitel __Quantum Ready__, waarbij in ieder hoofdstuk een stukje theorie achter deze proef wordt behandeld (licht, energie, fluorescentie, magnetisme).

Hieronder volgt eerst een beknopte theoretische toelichting op en daarna instructies voor het uitvoeren van de proef. 

# 2. Theorie

### 2.1 NV-centers in een diamantrooster
<div>
    <img src="./media/nv/nitrogen-vacancy.png" width="250" align="right"/>
</div>
<br>
Bron afbeelding: NIST, publiek domein (https://www.nist.gov/programs-projects/diamond-nv-center-magnetometry)
<br>

Diamant is opgebouwd uit koolstofatomen in een rooster met een zgn. face centered cubic structuur. Behale koolstofatomen kunnen er in het rooster ook defecten voorkomen, zoals gaten of andere atoomsoorten. Bij deze proef wordt gebruik gemaakt van stikstof-gat (nitrogen-vacancy of NV) defecten. Deze defecten kunnen in natuurlijke diamant voorkomen maar ook in het lab gemaakt worden. Kort door de bocht: de diamant groeit in een atmosfeer van stikstof gas, waardoor er stikstofatomen in het rooster verstrikt raken. Later worden er met gaten in het rooster geschoten, bijvoorbeeld met een elektronen bundel. Door de diamant te verhitten tot zo'n 900° Celsius migreren de gaten door het rooster heen naar de stikstofatomen. Zo ontstaat een stikstof-gat-paar (NV center). Diamanten met een hoge concentratie NV-centers zijn roze.

Er wijzen vijf ongebonden elektronen naar het gat: drie van de naburige koolstofatomen en twee van het stikstofatoom (stikstof staat één plaats rechts van koolstof in het periodiek systeem). Het blijkt energetisch gunstig om een zesde elektron in te vangen in het gat. Zo ontstaat een negatief geladen NV$^{-}$-center. In deze toestand kan het NV-center worden gebruikt als qubit of magneetsensor.


#### 2.1. 3D rooster bekijken
Wil je het diamantrooster met NV center beter bekijken, voer dan met onderstaande cel uit. Dit kan door rechtsboven op de raket te drukken (live code execution) en vervolgens op run te klikken.

In [ ]:
#%matplotlib qt

from ipywidgets import interact
from ipywidgets import IntSlider
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from itertools import product, combinations

coordinates = np.array([
    [0.5, 0, 0.0], [0, 0.5, 0.0], [1, 0.5, 0.0], [0.5, 1, 0.0],
    [0.25, 0.75, 0.25],
    [0, 0, 0.5], [1, 0, 0.5], [0, 1, 0.5], [1, 1, 0.5],
    [0.25, 0.25, 0.75], [0.75, 0.75, 0.75], 
    [0.5, 0, 1], [0, 0.5, 1], [1, 0.5, 1], [0.5, 1, 1],
])

#Set the location of the NV center
N = np.array([0.75, 0.25, 0.25])
V = np.array([0.5, 0.5, 0.5])

def plot_atoms(elev, azim):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot spheres with dark grey outline and lighter grey fill
    for coord in coordinates:
        if np.all(coord == [0.5, 0.5, 0.5]):
            ax.scatter(coord[0], coord[1], coord[2], color='none', edgecolor='black', linestyle='dashed', s=200)
        else:
            ax.scatter(coord[0], coord[1], coord[2], color='dimgrey', edgecolor='black', s=200)

    # Plot blue sphere for N and dashed circumference for V
    ax.scatter(N[0], N[1], N[2], color='blue', edgecolor='black', s=200)
    ax.scatter(V[0], V[1], V[2], color='w', edgecolor='black', s=200)

    # Function to calculate Euclidean distance between two points
    def distance(point1, point2):
        return np.linalg.norm(point1 - point2)

    # Function to find nearest neighbors for each atom
    def find_nearest_neighbors(coords, threshold):
        nearest_neighbors = []
        for i, atom1 in enumerate(coords):
            for j, atom2 in enumerate(coords):
                if j > i:  # Avoid checking distances twice
                    if distance(atom1, atom2) <= threshold:
                        nearest_neighbors.append((i, j))
        return nearest_neighbors

    # Find nearest neighbors among the atoms
    nearest_neighbors = find_nearest_neighbors(np.concatenate((coordinates, [V], [N]), axis=0), threshold=0.6)  # Adjust threshold as needed

    # Plot lines between nearest neighbors
    for pair in nearest_neighbors:
        atom1 = np.concatenate((coordinates, [V], [N]), axis=0)[pair[0]]
        atom2 = np.concatenate((coordinates, [V], [N]), axis=0)[pair[1]]
        if np.any(atom1 == V) or np.any(atom2 == V) or np.any(atom1 == N) or np.any(atom2 == N):
            # Plot dashed lines for connections involving the NV centers
            ax.plot([atom1[0], atom2[0]], [atom1[1], atom2[1]], [atom1[2], atom2[2]], color='green', linestyle='dashed')
        else:
            ax.plot([atom1[0], atom2[0]], [atom1[1], atom2[1]], [atom1[2], atom2[2]], color='green')

    # Plot dashed cube around data points
    for s, e in combinations(np.array(list(product([0, 1], repeat=3))), 2):
        if np.sum(np.abs(s-e)) == 1:
            ax.plot3D(*zip(s, e), color="b", linestyle='dashed')

    ax.view_init(elev=elev, azim=azim)
    ax.set_proj_type('ortho')
    ax.set_box_aspect([1,1,1])  # Set aspect ratio
    
    plt.axis('off')

    plt.show()

plot_atoms(12,82)


### 2.2 Spintoestanden en energieniveaus
Elektronen hebben een spin van ±½, vaak spin down (↓) en spin up (↑) genoemd. De elektronen in het NV-center zijn met elkaar verstrengeld en gedragen zich gezamenlijk als een systeem met spin 1. Dit levert (netto) de volgende toestanden op:

Een singlet toestand: $\frac{1}{\sqrt{2}}$ (↓↑ − ↑↓)

Drie triplet toestanden:
*  $\frac{1}{\sqrt{2}}$ (↓↑ + ↑↓)
*  ↑↑
*  ↓↓

__Vraag:__ wat zijn de spin-waardes van ieder van deze vier toestanden?
 
In het onderstaande energieniveauschema zijn de grondtoestand en de aangeslagen niveaus van de triplet toestanden in de paarse vlakken aangegeven. De singlet toestand is in een geel vlak afgebeeld. Zoals je kunt zien is de energie van de singlet toestand hoger in energie dan de triplet-grondtoestanden. (N.B.: de overgangen tussen de toestanden kun je voor nu negeren, deze worden in 2.3 toegelicht).

<div>
<img src="./media/nv/NV-energy-levels.svg" width="350"/>
</div>
<center>(Diagram door BonPhire, BY-SA 4.0, https://en.wikipedia.org/wiki/File:NV-energy-levels.svg)</center>

Zonder extern magnetisch veld hebben de energieniveaus van de ↑↑-toestand (m$_{s}$ = +1) en de ↓↓-toestand (m$_{s}$ = −1) dezelfde energie. Is er wel een extern magneetveld, dan splitsen deze energieniveaus op: dit wordt zeemansplitsing genoemd (naar Pieter Zeeman). De mate van opsplitsing hangt af van de sterkte van het magneetveld in de richting van het NV-centrum.

__Vraag__: waarom verandert de laagste energietoestand niet onder invloed van een magnetisch veld?

Hoe sterker het magnetisch veld, hoe dichter de energie van de m$_{s}$=-1 toestand bij de grondtoestand komt te liggen. Bij een magnetisch veld van 102.5 mT snijden de twee energieniveaus elkaar en bij een nog groter veld wordt dus de m$_{s}$=-1 toestand de grondtoestand. Dit is weergegeven in onderstaand diagram.

<img src="./media/nv/mixing.png" width="550"/>
<center>(Diagram door Patel et al., https://arxiv.org/abs/2306.17531)</center>

Doordat de scheiding tussen de energietoestanden niet heel groot is (zeker in vergelijking met k$_{B}$T bij kamertemperatuur) zul je in (bij benadering) een superpositie mogen verwachten tussen de m$_{s}$=0 en m$_{s}$=-1 toestanden. Met het aanleggen van een extern magneetveld wordt de bijdrage van de m$_{s}$=-1 toestand in die superpositie steeds groter.





### 2.3 Fluorescentie
Een NV center fluoresceert onder invloed van groen (laser)licht. Dat is op onderstaande foto goed te zien.Je ziet een buisje met diamantpoeder met een hoge concentratie NV centers (3.5 ppm). Rechts wordt het buisje beschenen met een groene laser. De fluorescentie is met het blote oog te zien.

<img src="./media/nv/Fluorescentie.jpg" width="450"/>

Vanuit zowel de m$_{s}$ = 0 als vanuit de m$_{}s$ = ±1 toestand wordt het systeem door het groene licht aangeslagen naar een band van energietoestanden. Omdat de spin behouden is verliest het systeem eerst energie via niet-radiatieve, interne overgangen en komt het terecht in de aangeslagen triplettoestand met overeenkomend m$_{s}$-getal. Van daaruit valt het terug naar de grondtoestand onder uitzending van een rood foton met een golflengte van tussen de 650 en 750 nm (fluorescentie). Deze overgangen zijn weergegeven met de dikke groene en rode pijlen in figuur 1.

<img src="./media/nv/NV-transitions.svg" width="400"/>
(Diagram door BonPhire, BY-SA 4.0, https://commons.wikimedia.org/wiki/File:NV-transitions.svg)

Er is daarnaast een kleine kans dat het NV center terugvalt via de tussengelegen singlet grondtoestand van de singlet toestand ($^{1}A_{1}$ in de figuur). In dit geval is spin niet behouden en dat maakt deze overgangen minder waarschijnlijk. Bij dit proces komt geen (zichtbaar) licht vrij. Dit niet-radiatieve verval is waarschijnlijker vanuit de m$_{s}$ = ±1 toestand dan vanuit de m$_{s}$ = 0 toestand. Zoals in 2.2 is beschreven neemt bij de kans, dat het NV center zich in de m$_{s}$ = -1 toestand bevindt, toe en daarmee neemt de hoeveelheid fluorescentie af. Zo geeft de mate van fluorescentie informatie over de spintoestand waarin het NV centrum zich bevindt en daarmee kan in principe de grootte van het extern magnetisch veld worden bepaald.

# 3. Proefopstelling



### 3.1 Kastje
<div></div>
De proefopstelling bestaat uit een lichtdicht doosje. In het doosje zie je aan het ene uiteinde een rood plaatje acrylaat. Achter het acrylaat bevindt zich een TSL2591 sensor voor Arduino, die met een fotodiode lichtintensiteit meet. Deze heeft een heel groot dynamisch bereik. Dat wil zeggen dat hij zeer gevoelig is voor (veranderingen) in lichtintensiteit. Het nadeel hiervan is dat de sensor (relatief) langzaam (hooguit 8 metingen per seconde).

<div>
    <img src="./media/nv/LaserTSL2.jpeg" width="200"/>
</div>

Aan de voorkant van het acrylaat is diamantpoeder met een hoge concentratie NV-centers gelijmd m.b.v. secondelijm. Vanaf de opening aan de andere kant van het doosje schijn je __STRAKS__ met een groene laser op het diamantpoeder.

<div>
    <img src="./media/nv/Laser.jpeg" width="200"/>
</div>

__Vraag:__ waarom is het acrylaat rood?

Het kastje zal niet 100% licht dicht zijn. Om toch zo weinig mogelijk achtergrondsignaal te hebben kun je de zijdes van het kastje nog even goed samendrukken.


### 3.2 Laser
We gebruiken een 520 nm laser van Thorlabs met een USB-aansluiting. Wacht nog even met aansluiten. Steek de laser door het gat in het kastje en klem de laser vast met het extra strakke plaatje dat los in de behuizing licht. Doe (nog) niks met het dubbelzijdig tape dat op het plaatje zit. De laser moet worden uitgelijnd op het diamantpoeder. Met de laser in het doosje sluit je de USB-kabel aan op een vrije poort in de computer (of een andere USB-spanningsbron). De laser gaat pas aan enkele seconden nadat je op het zwarte knopje op de USB-stekker hebt geklikt.

Probeer de laser zo goed mogelijk te richten op het diamantpoeder. Door de hoge brekingsindex van diamant zie je de korrels duidelijk oplichten / schitteren. Staat de laser goed uitgelijnd? Dan kun je het houten plaatje eventueel vastplakken met het dubbelzijdig tape, maar dat hoeft niet. Lukt het niet om de laser goed op de korrels te richten zijn er (negatieve) lensjes aanwezig om de laserbundel te verbreden. De intensiteit van het fluorescentiesignaal (straks) neemt hierdoor wel af.

__Zet de laser weer UIT__: we gaan zo eerst experimenteren met een donker doosje. De laser moet het gat afsluiten, maar staat niet aan.

### 3.3 Dekseltje
Doe nu het dekseltje op het doosje om de boel af te sluiten. Dekseltje steekt bewust wat extra uit om schaduw te bieden. Als je de deksel weer van de doos haalt, koppel dan de Arduino (zie 3.4) even af. Anders moet je mogelijk lang wachten voordat de sensor weer stabiel is.

### 3.4 Arduino
De Arduino is al voorgeprogrammeerd en hoeft in principe alleen te worden aangesloten op de computer met een USB-kabel. Wel moet bekend zijn welke COM-poort aan de Arduino is toegewezen. In de Python code kan de juiste COM-poort worden ingevoerd.

De draden van de TSL2591 steken aan de zijkant uit het doosje. Deze moeten, op volgorde, als volgt worden aangesloten op de Arduino:

*  Geel in SCL of A5 bij een UNO
*  Blauw in SDA of A4 bij een UNO
*  Zwart in GND
*  Rood in 5V

### 3.5 Vooruitblik verdere ontwikkeling
<div>
    <img src="./media/nv/Nederlands_verkeersbord_J16.svg" width="100" align="right"/>
</div>
Voor we beginnen met meten benadrukken we dat de huidige proefopstelling _work in progress_ is. In de komende maanden worden nog verschillende verbeteringen verwacht, waaronder:

*  Kwantitatief bepalen van magnetisch veld / Zeeman-splitsing m.b.v. resonantie met een RF-golf
*  Verbeterde lichtdichtheid, o.a. door een groenfilter toe te voegen voor het roodfilter.
*  Kleiner maken van de opstelling met betere uitlijning laser
*  Aansturing via *stand alone* software / GUI

De opstelling is vanaf de zomer beschikbaar in het *Quantum Ready*-Lab bij de TU Delft voor profielwerkstukken havo (!) en vwo. Op dit moment wordt er gewerkt aan een NLT-module over quantum sensoren waar deze proef ook onderdeel van zal zijn. Voor meer informatie kun je contact opnemen met Rutger Ockhorst via onz@tudelft.nl of (voor leerlingen) met Lennard Duynkerke via scholierenlab@tudelft.nl



# 4. Experimenteren
Metingen kun je opstarten vanuit hoofdstuk 5. Voel je vrij om te experimenteren, maar we doen hieronder een paar suggesties om je op weg te helpen.

Voordat je begint met meten: controleer steeds of je een stabiel signaal hebt. Soms kan de sensor enige tijd nodig hebben om te stabiliseren, bijvoorbeeld als het doosje net is open geweest en de sensor (over)belicht is.

### 4.1 Gevoeligheid voor achtergrondlicht bepalen
Voer een meting uit van 60 seconden met een dicht doosje, laser uit. Dit zou een signaal van 0 tot ongeveer 100 op mogen leveren. Lager is beter. Kijk of het kastje erg gevoelig is voor variaties in de omgeving. Hou je handen bijvoorbeeld eens om het kastje heen of schuif je stoel eens naar achter. In het ideale geval heeft dit geen invloed.

### 4.2 Fluorescentiesignaal zonder extern magnetisch veld bepalen
Zet de laser aan en doe een meting van 30 seconden. bij een goed uitgelijnde laser kun je een fluorescentiesignaal verwachten van 10k tot 20k. Heb je de negatieve lens gebruikt dan is 2k tot 4k te verwachten. Controleer of het signaal stabiel is (fluctuaties van minder dan een procent zijn ok). Het kan soms voorkomen dat de sensor langer nodig heeft om te stabiliseren. Dan zie je bijvoorbeeld een continu oplopend signaal. Herhaal de meting dan een aantal keer, misschien na stabiliseren van de laser of verplaatsen van het kastje, tot je een stabiel signaal ziet.

### 4.3 Invloed van het magnetisch veld waarnemen
<img src="./media/nv/magnet.png"/>
<br>
Als je een stabiel signaal hebt kun je een meting doen met een extern magnetisch veld. Let op: de magneten die we hier gebruiken zijn sterk tot __zeer__ sterk. Houd je pinpas, laptop etc. dus uit de buurt. Controleer je werkplek ook op losliggende metalen onderdelen die door de magneet aangetrokken kunnen worden (misschien met je hand er tussen: auw!). Ben je klaar met de magneet, berg 'm dan op in de doos, laat ze absoluut niet los op tafel liggen.

Er is een klein podium beschikbaar waar je de magneet op kunt leggen. De magneet ligt dan precies in lijn met de laser en het diamantpoeder. Zet je de magneet tegen de achterkant van het kastje aan dan is de afstand tot het poeder 1,5 cm. Er is op het podiumpje een schaalverdeling zichtbaar die begint bij 1,5 cm en oploopt in stappen van 0,5 cm. Onderzoekjes die je zou kunnen doen:

*  hoe verandert het fluorescentiesignaal met de afstand tussen magneet en diamantpoeder?
*  welk van de beschikbare magneten is sterker?
*  ...

# 5. Meten

### 5.1 Meting met opstelling
In de cel hieronder wordt een Python-programma aangeroepen waarmee, via de Arduino, lichtsterkte kan worden gemeten als functie van tijd en in een grafiek kan worden weergegeven. Op de y-as staat de ruwe data van de sensor, dus in zeker opzicht _arbitrary units_, maar in principe is dit om te rekenen naar de lichtsterkte in lux. De sensor kan signalen tussen de 188 μlux en 88 klux meten. Let er op dat de juiste COM-poort moet zijn geselecteerd. Meestal gaat dit "automagisch". 

Let op: als er een meting aan de gang is kun je die niet zo maar stoppen. Dat wil zeggen: als je tussentijds op de stop-knop drukt moet je de kernel opnieuw opstarten. Bovendien is de verbinding met de Arduino / COM-poort dan niet netjes afgesloten. Dat los je op door de USB-stekker uit de computer te halen en er opnieuw in te steken.

De **software**, compleet met Arduino code en de Python Notebook file is ook te downloaden in een zipfile: {Download}`software<./media/nv/DiamantNVcenter.zip>`. Handig als de cel hieronder niet werkt.

In [10]:
# Dit werkt alleen als je de opstelling hebt aangesloten! Anders natuurlijk niet. 
# In de volgende cel zie je een voorbeeldmeting.
import os
import sys

# Get the absolute path of the current notebook
notebook_path = os.path.abspath("nv.ipynb")

# Get the directory of the current notebook
notebook_dir = os.path.dirname(notebook_path)

# Change the current working directory to the notebook directory
os.chdir(notebook_dir)

# Check the current working directory
print("Current working directory:", os.getcwd())

# Verify the file path
file_path = './media/nv/interactive-NV-latest.py'
if os.path.exists(file_path):
	%run $file_path
else:
	print(f"File not found: {file_path}")

### 5.2 Voorbeeld van een meting
Hieronder een voorbeeld van een meetreeks. Verticaal staat (in willekeurige eenheid) de intensiteit van de fluorescentie. Rond 10 seconden wordt er een magneet bij het poeder gebracht en gehouden. Rond 22 seconden wordt de magneet weer (rustig) weg gehaald.
<img src="./media/nv/oorbeeldMeting.png" width="400"/>

# 6. Feedback
Het experiment dat je zojuist hebt uitgevoerd is dus nog in ontwikkeling. We zijn heel benieuwd naar je ervaring. Als je feedback wil geven, vul dan het volgende Google Form in:

https://forms.gle/wxKwAghYxAZhXb6N9